In [23]:
import keras 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

#Splitting the data between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [24]:
num_classes = 0

#Adding one more dimension to reshape the matrix
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

#Converting class vectors to binary class metrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [25]:
#Creating the model
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])

In [26]:
#Training the model
hist = model.fit(x_train, y_train,
                 batch_size = batch_size, 
                 epochs = epochs, 
                 verbose = 1, 
                 validation_data = (x_test, y_test))
print("The model has successfully trained")

model.save('mnist.h5')
print("Saved the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 156s 332ms/step - loss: 2.2923 - accuracy: 0.1252 - val_loss: 2.2631 - val_accuracy: 0.3124
Epoch 2/10
469/469 [==============================] - 160s 342ms/step - loss: 2.2463 - accuracy: 0.2477 - val_loss: 2.2080 - val_accuracy: 0.5367
Epoch 3/10
469/469 [==============================] - 156s 332ms/step - loss: 2.1878 - accuracy: 0.3676 - val_loss: 2.1334 - val_accuracy: 0.6284
Epoch 4/10
469/469 [==============================] - 155s 332ms/step - loss: 2.1044 - accuracy: 0.4652 - val_loss: 2.0252 - val_accuracy: 0.7004
Epoch 5/10
469/469 [==============================] - 160s 341ms/step - loss: 1.9858 - accuracy: 0.5378 - val_loss: 1.8683 - val_accuracy: 0.7488
Epoch 6/10
469/469 [==============================] - 154s 329ms/step - loss: 1.8184 - accuracy: 0.5955 - val_loss: 1.6532 - val_accuracy: 0.7817
Epoch 7/10
469/469 [==============================] - 156s 334ms/step - loss: 1.6035 - accuracy: 0.6381 - val_loss: 1.3952 -

In [27]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8039253950119019
Test accuracy: 0.836899995803833


In [28]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):

    #Resizing image to 28x28 px
    img = img.resize((28,28))

    #Converting rgb to grayscale
    img = img.convert('L')
    img = np.array(img)

    #Reshaping to support our model input and normalizing
    img = img.reshape(1, 28, 28, 1)
    img = img/255.0

    #Predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        #Creating elements
        self.canvas = tk.Canvas(self, width = 300, height = 300, bg = "white", cursor = "cross")
        self.label = tk.Label(self, test = "Thinking!", font = ("Helvetica", 48))
        self.classify_btn = tk.Button(self, test = "Recognise", command = self.classify_handwriting)
        self.button_clear = tk.Button(self, test = "Clear", command = self.clear_all)

        #Grid Structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

        def clear_all(self):
            self.canvas.delete("all")

        def classify_handwriting(self):
            HWND = self.canvas.winfo_id() #Getting the handle of canvas
            rect = win32gui.GetWindowRect(HWNF) #Getting the coordinate of canvas
            im = ImageGrab.grab(rect)

            digit, acc = predict_digit(im)
            self.label.configure(text = str(digit) + ', ' + str(int(acc*100)) + '%')

            def draw_lines(self, event):
                self.x = event.x
                self.y = event.y
                r = 8
                self.canvas.create_oval(self.x-r, self.y-r, self.x+r, self.y+r, fill = 'black')

app = App()
mainloop()

ModuleNotFoundError: ignored